<a href="https://colab.research.google.com/github/camilobrownpinilla/PruningExplainability/blob/main/geometry%5CHessian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from torch import nn, optim
from torch.autograd import grad
from transformers import BertForSequenceClassification, BertTokenizerFast
from datasets import load_dataset

# Disable efficient attention kernels
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_math_sdp(True)

# Configuration
model_name = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_random_directions = 3
batch_size = 4

# Load dataset and tokenizer
dataset = load_dataset("glue", "sst2", split="train").shuffle(seed=42)
dataset = dataset.select(range(100))  # smaller subset
tokenizer = BertTokenizerFast.from_pretrained(model_name)

def encode_fn(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=32)

encoded_dataset = dataset.map(encode_fn, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
loader = torch.utils.data.DataLoader(encoded_dataset, batch_size=batch_size)

# Load model after disabling efficient attention
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
model.eval()

# Get a batch
batch = next(iter(loader))
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
labels = batch['label'].to(device)

# Forward pass
outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
loss = outputs.loss

params = [p for p in model.parameters() if p.requires_grad]
grad_params = grad(loss, params, create_graph=True)

# Flatten gradients
flat_grad = torch.cat([g.reshape(-1) for g in grad_params])
param_size = sum(p.numel() for p in params)
approx_hessian_diag = torch.zeros(param_size, device=device)

for _ in range(num_random_directions):
    v = torch.randn(param_size, device=device)
    dot = torch.dot(flat_grad, v)
    hvp = grad(dot, params, retain_graph=True)
    hvp_flat = torch.cat([h.reshape(-1) for h in hvp])
    approx_hessian_diag += hvp_flat * v

approx_hessian_diag /= num_random_directions
print("Approximate Hessian diagonal:", approx_hessian_diag)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Approximate Hessian diagonal: tensor([ 0.0000,  0.0000,  0.0000,  ..., -1.3061,  0.6348, -3.5372])
